## Extinction, galactic latitude and peak absolute magnitude

### To run this notebook, please [follow the instructions](https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html) or else it won`t work.
The instructions are at https://lasair-lsst.readthedocs.io/en/main/core_functions/python-notebooks.html

This notebook computes three Lasair features: extinction E(B-V), galactic latitude, 
and peak extinction-corrected absolute magnitude (PECAM).

In [48]:
import sys
sys.path.append('../API_lsst')
import math, json, redshift_distance

### 1. Extinction
#### Get the dustmaps package set up. 
We use this package to compute the extinction from the dustmap of 
[Schlegel, Finkbeiner, and Davis](https://iopscience.iop.org/article/10.1086/305772).
Once the package is installed, the dustmaps themselves must be downloaded with fetch() -- change the
directory name according to your own setup.
Note that the fetch() method only needs to be called once. 

In [49]:
# Uncomment this to install the dustmaps package
#!/usr/bin/pip3 install dustmaps
from dustmaps.config import config

# Set this to where you want the two 64 Mbyte dustmap files
config['data_dir'] = '/Users/rwillia5/Desktop'
import dustmaps.sfd

# Uncomment the following to download the SFD dustmap
# dustmaps.sfd.fetch()

from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord
sfd = SFDQuery()

#### Corrected colour for extinction
These colour corrections are multiplied by the extinction E(B-V) to get a magnitude correction. 
They arefrom Table 6 of 
[Schlafly and Finkbeiner](https://iopscience.iop.org/article/10.1088/0004-637X/737/2/103) with RV=3.1

In [50]:
# The LSST bands
EXTCOEF   = {'u':4.145, 'g':3.237, 'r':2.273, 'i':1.684, 'z':1.323, 'y':1.088}

# Modify magnitude for extinction as
# mag - ebv*EXTCOEF[band]

#### Extinction from sky position

In [51]:
def computeExtinction(ra, decl):
    c = SkyCoord(ra, decl, unit="deg", frame='icrs')
    ebv = float(sfd(c))
    return ebv

### 2. Magnitude and Flux
#### Transform between magnitudes and nanoJanskies

In [52]:
def mag2flux(mag):
    # flux in nanoJ
    flux =  math.pow(10, (31.4 - mag)/2.5)
    return flux
def flux2mag(flux):
    # flux in nanoJ
    mag = 31.4 - 2.5*math.log10(flux)
    return mag

### 3. Peak extinction-corrected magnitude
From the E(B-V) and lightcurve in flux, we compute magnitude, 
correct them with extinction, then find the brightest point, and the associated band.

In [53]:
def findPeakExtMag(ebv, z, lightcurve):
    peakMag  = 100
    peakBand = None
    for diaSource in lightcurve:
        mjd        = diaSource['midpointMjdTai']
        band       = diaSource['band']
        psfFlux    = diaSource['psfFlux']
        psfFluxErr = diaSource['psfFluxErr']
        if psfFlux > 0:
            mag = flux2mag(psfFlux)
            # extinction correction and k-correction
            correctedMag = mag - ebv*EXTCOEF[band] + 2.5*math.log(1+z)
            if correctedMag < peakMag:
                peakMag = correctedMag
                peakBand = band
    return (peakMag, peakBand)

### 4. Galactic latitude

In [54]:
# https://en.wikipedia.org/wiki/Galactic_coordinate_system
def galacticLat(ra, decl):
    alphaNGP = 192.85948
    deltaNGP =  27.1283
    sdngp = math.sin(math.radians(deltaNGP))
    cdngp = math.cos(math.radians(deltaNGP))
    sde = math.sin(math.radians(decl))
    cde = math.cos(math.radians(decl))
    cra = math.cos(math.radians(ra - alphaNGP))
    glat = math.degrees(math.asin(sdngp*sde + cdngp*cde*cra))
    return glat

### 5. Test run
Use the Lasair API to find some objects with a host galaxy, then compute their features.
Make sure to connect to the right endpoint.

In [55]:
#!/usr/bin/pip3 install lasair
from lasair import LasairError, lasair_client as lasair
import settings
endpoint = "https://api.lasair.lsst.ac.uk/api"
!mkdir cache
L = lasair(settings.API_TOKEN, endpoint=endpoint)

mkdir: cache: File exists


#### Get some working objects with a Sherlock redshift

In [56]:
selected = """ 
  objects.diaObjectId, objects.ra, objects.decl, 
  sherlock_classifications.z, sherlock_classifications.photoz
"""
tables = 'objects,sherlock_classifications'
conditions = """
  classification="SN" AND 
  (sherlock_classifications.z > 0 OR sherlock_classifications.photoz > 0)
"""
results = L.query(selected, tables, conditions, limit = 10)
for row in results:
    objectId = row['diaObjectId']
    ra       = row['ra']
    decl     = row['decl']
    z        = row['z']
    photoz   = row['photoz']
    if not z:
        z = photoz
    object   = L.object(objectId, lasair_added=False, lite=True)

    # compute galactic latitude
    ebv = computeExtinction(ra, decl)
    
    # compute extinction
    galLat = galacticLat(ra, decl)

    # compute peak extinction-corrected apparent magnitude
    (peakMag, peakBand) = findPeakExtMag(ebv, z, object['diaSourcesList'])
    if not peakBand:
        continue

    # combine z and apparent mag to get absolute mag
    # using Ken Smith code from Atlas for distance modulus
    distances = redshift_distance.redshiftToDistance(z)
    distanceModulus = distances['dmod']

    absMag = peakMag - distanceModulus

    print('%s: ra=%.2f decl=%.2f z=%.3f E(B-V)=%.2f peakMax=%.2f peakBand=%s, absMag=%.2f' % 
          (objectId, ra, decl, z, ebv, peakMag, peakBand, absMag))


169298432615252003: ra=10.90 decl=-43.78 z=0.020 E(B-V)=0.01 peakMax=21.93 peakBand=z, absMag=-12.80
169298433044119587: ra=10.89 decl=-43.81 z=0.020 E(B-V)=0.01 peakMax=22.38 peakBand=i, absMag=-12.36
169298436674813978: ra=8.80 decl=-43.99 z=0.025 E(B-V)=0.01 peakMax=21.61 peakBand=i, absMag=-13.58
169302852818698254: ra=10.90 decl=-45.16 z=0.013 E(B-V)=0.01 peakMax=22.83 peakBand=i, absMag=-10.84
169302852856971293: ra=9.29 decl=-43.54 z=0.045 E(B-V)=0.01 peakMax=22.69 peakBand=i, absMag=-13.81
169302854317113355: ra=9.38 decl=-44.14 z=0.052 E(B-V)=0.01 peakMax=21.18 peakBand=z, absMag=-15.64
169302854334939154: ra=8.60 decl=-43.67 z=0.020 E(B-V)=0.01 peakMax=20.49 peakBand=z, absMag=-14.19
169302854850838530: ra=9.38 decl=-44.14 z=0.052 E(B-V)=0.01 peakMax=20.97 peakBand=i, absMag=-15.85
169302869841281049: ra=8.96 decl=-43.50 z=0.023 E(B-V)=0.01 peakMax=20.47 peakBand=r, absMag=-14.57
169342390655516729: ra=8.24 decl=-43.09 z=0.159 E(B-V)=0.01 peakMax=23.94 peakBand=g, absMag=-15.